# 批量酿造提取系数分析器

## 功能说明

本脚本用于批量读取 brew_data 目录下的所有酿造记录，并计算以下指标：

1. **洗糟前（糖化）提取系数 (Pre-Sparge Extraction Coefficient)**：糖化结束后头道麦汁的提取系数
2. **洗糟后提取系数 (Post-Sparge Extraction Coefficient)**：收集完洗糟水后满锅麦汁的提取系数
3. **熬煮后效率 (Post-Boil Efficiency)**：入发酵罐前最终麦汁的效率

### 计算公式

$$ \text{提取系数} = \frac{M_{\text{Actual Extract}}}{M_{\text{Potential Extract}}} $$

其中：
- $M_{\text{Potential Extract}} = \text{总麦芽重量} \times \text{潜在浸出率}$
- $M_{\text{Actual Extract}} = \text{麦汁质量} \times \text{Plato} / 100 = \text{水量} \times SG \times \frac{\text{Plato}}{100}$
- $\text{Plato} \approx (\text{SG} - 1.000) \times 250$

**注意**：
- 洗糟前提取系数：使用糖化锅初始水量（strike_water）计算
- 洗糟后提取系数：使用总水量（strike_water + sparge_water）计算，而非麦汁体积，因此可能大于1

In [28]:
import os
import sys
import yaml
import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional

# 配置 pandas 显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 定义数据目录 - 使用当前工作目录
BREW_DATA_DIR = Path.cwd() / 'brew_data'

print(f"📁 数据目录: {BREW_DATA_DIR}")
print(f"📁 目录是否存在: {BREW_DATA_DIR.exists()}")

def load_brew_records(data_dir: Path) -> List[Dict]:
    """加载所有酿造记录 YAML 文件"""
    records = []
    yaml_files = sorted(data_dir.glob('*.yaml'))
    
    print(f"📄 找到 {len(yaml_files)} 个 YAML 文件")
    
    for yaml_file in yaml_files:
        with open(yaml_file, 'r', encoding='utf-8') as f:
            record = yaml.safe_load(f)
            records.append(record)
    
    return records

print("✅ 数据加载函数定义完成！")

📁 数据目录: /Users/zhangle/SynologyDrive/IDC/IDC_notebook/BrewMetrics/brew_data
📁 目录是否存在: True
✅ 数据加载函数定义完成！


In [29]:
# 提取系数计算函数

def sg_to_plato(sg: float) -> float:
    """将比重(SG)转换为糖度(Plato)"""
    return (sg - 1.000) * 1000 / 4

def calculate_potential_extract(total_malt_weight: float, potential_extract: float) -> float:
    """计算麦芽的理论浸出物总量"""
    return total_malt_weight * potential_extract

def calculate_actual_extract(water_volume_l: float, sg: float) -> Optional[float]:
    """计算实际收集到的浸出物质量
    
    Args:
        water_volume_l: 水量（升）
        sg: 麦汁比重
    
    Returns:
        浸出物质量（克）
    """
    if water_volume_l is None or sg is None:
        return None
    
    wort_mass = water_volume_l * sg  # 麦汁质量 = 水量 × 比重
    plato = sg_to_plato(sg)
    actual_extract = wort_mass * (plato / 100)
    return actual_extract

def calculate_extraction_coefficient(total_malt_weight: float, 
                                     potential_extract: float,
                                     water_volume_l: float, 
                                     sg: float) -> Optional[float]:
    """计算提取系数
    
    Args:
        total_malt_weight: 总麦芽重量（kg）
        potential_extract: 潜在浸出率
        water_volume_l: 水量（升）- 用于提取系数计算的水量
        sg: 麦汁比重
    
    Returns:
        提取系数（小数形式，如0.75表示75%）
    """
    if water_volume_l is None or sg is None:
        return None
    
    potential = calculate_potential_extract(total_malt_weight, potential_extract)
    actual = calculate_actual_extract(water_volume_l, sg)
    
    if potential == 0 or actual is None:
        return None
    
    coefficient = actual / potential
    return round(coefficient, 4)

print("✅ 计算函数定义完成！")

✅ 计算函数定义完成！


In [30]:
def calculate_stage_efficiencies(record: Dict) -> Dict:
    """计算单个记录各阶段的提取系数/效率"""
    metadata = record.get('metadata', {})
    ingredients = record.get('ingredients', {})
    measurements = record.get('measurements', {})
    mash = record.get('mash', {})
    
    # 获取process_logs中最后一次测量的PH值
    process_logs = mash.get('process_logs', [])
    final_ph = None
    if process_logs and len(process_logs) > 0:
        final_ph = process_logs[-1].get('ph')
    
    result = {
        'date': metadata.get('date'),
        'recipe_name': metadata.get('recipe_name'),
        'batch_id': metadata.get('batch_id'),
        'total_malt_weight': ingredients.get('total_malt_weight'),
        'potential_extract': ingredients.get('potential_extract'),
        'notes': ingredients.get('notes', ''),
        'strike_water': mash.get('strike_water'),
        'water_grain_ratio': mash.get('water_grain_ratio'),
        'sparge_water': mash.get('sparge_water'),
        'final_ph': final_ph,
        'target_temperature': mash.get('target_temperature')
    }
    
    total_malt = ingredients.get('total_malt_weight')
    potential = ingredients.get('potential_extract')
    strike_water = mash.get('strike_water')
    sparge_water = mash.get('sparge_water')
    
    # 计算洗糟前（糖化）提取系数
    # 使用糖化锅初始水量（strike_water）计算
    pre_sparge = measurements.get('pre_sparge', {})
    result['pre_sparge_volume'] = pre_sparge.get('volume')  # 原始测量数据（仅供参考）
    result['pre_sparge_sg'] = pre_sparge.get('gravity')
    result['pre_sparge_coefficient'] = calculate_extraction_coefficient(
        total_malt, potential, strike_water, pre_sparge.get('gravity')
    )
    
    # 计算洗糟后提取系数
    # 使用总水量（strike_water + sparge_water）计算，而非麦汁体积
    pre_boil = measurements.get('pre_boil', {})
    total_water_volume = None
    if strike_water is not None and sparge_water is not None:
        total_water_volume = strike_water + sparge_water
    
    result['pre_boil_volume'] = pre_boil.get('volume')  # 原始测量数据（仅供参考）
    result['pre_boil_sg'] = pre_boil.get('gravity')
    result['post_sparge_coefficient'] = calculate_extraction_coefficient(
        total_malt, potential, total_water_volume, pre_boil.get('gravity')
    )
    
    # 计算熬煮后效率（使用实际麦汁体积）
    post_boil = measurements.get('post_boil', {})
    result['post_boil_volume'] = post_boil.get('volume')
    result['post_boil_sg'] = post_boil.get('gravity')
    result['post_boil_efficiency'] = calculate_extraction_coefficient(
        total_malt, potential, post_boil.get('volume'), post_boil.get('gravity')
    )
    
    # 计算熬煮蒸发量
    if result['pre_boil_volume'] is not None and result['post_boil_volume'] is not None:
        result['boil_off'] = result['pre_boil_volume'] - result['post_boil_volume']
    else:
        result['boil_off'] = None
    
    # 计算谷物吸收率
    # 公式：(strike_water + sparge_water - pre_boil_volume) / total_malt_weight
    if (strike_water is not None and sparge_water is not None and 
        result['pre_boil_volume'] is not None and total_malt is not None and total_malt > 0):
        result['grain_absorption_rate'] = ((strike_water + sparge_water - result['pre_boil_volume']) / total_malt)
    else:
        result['grain_absorption_rate'] = None
    
    return result

print("✅ 数据处理函数定义完成！")

✅ 数据处理函数定义完成！


In [31]:
# 加载所有记录
records = load_brew_records(BREW_DATA_DIR)

print(f"\n📊 成功加载 {len(records)} 条酿造记录")
print("="*80)

# 计算所有记录的提取系数/效率
all_efficiencies = []
for record in records:
    try:
        efficiency_data = calculate_stage_efficiencies(record)
        all_efficiencies.append(efficiency_data)
    except Exception as e:
        batch_id = record.get('metadata', {}).get('batch_id', 'Unknown')
        print(f"⚠️  处理批次 {batch_id} 时出错: {e}")

print(f"\n✅ 成功计算 {len(all_efficiencies)} 条记录的提取系数/效率数据")

📄 找到 11 个 YAML 文件

📊 成功加载 11 条酿造记录

✅ 成功计算 11 条记录的提取系数/效率数据


In [32]:
# 创建 DataFrame 用于显示和统计分析
df = pd.DataFrame(all_efficiencies)

# 创建显示用的表格（保留数值型数据用于渐变着色）
display_df = pd.DataFrame({
    '日期': df['date'],
    '配方': df['recipe_name'],
    '麦芽重量': df['total_malt_weight'],
    '糖化水': df['strike_water'],
    '水料比': df['water_grain_ratio'],
    'PH值': df['final_ph'],
    '糖化温度': df['target_temperature'],
    '洗糟前提取系数': df['pre_sparge_coefficient'],
    '洗糟水': df['sparge_water'],
    '洗糟后提取系数': df['post_sparge_coefficient'],
    '谷物吸收率': df['grain_absorption_rate'],
    '熬煮蒸发量': df['boil_off'],
    '熬煮后效率': df['post_boil_efficiency']
})

# 显示表格
print(f"\n{'='*100}")
print(f"{'📋 酿造提取系数/效率汇总表'.center(100)}")
print(f"{'='*100}\n")

# 使用美化的样式
styled_df = (display_df.style
            .set_properties(**{'text-align': 'center', 'padding': '10px', 'color': '#495057', 'background-color': 'white'})
            .set_table_styles([{
                'selector': 'th',
                'props': [('background-color', '#6c757d'),
                         ('color', 'white'),
                         ('font-weight', 'bold'),
                         ('text-align', 'center'),
                         ('border', '1px solid #dee2e6'),
                         ('padding', '10px')]
            }, {
                'selector': 'td',
                'props': [('text-align', 'center'),
                         ('border', '1px solid #dee2e6'),
                         ('padding', '10px'),
                         ('color', '#495057'),
                         ('background-color', 'white')]
            }, {
                'selector': 'tr:hover',
                'props': [('background-color', '#e9ecef')]
            }])
            .format({
                '麦芽重量': '{:.2f}',
                '糖化水': '{:.1f}',
                '水料比': '{:.2f}',
                'PH值': '{:.2f}',
                '糖化温度': '{:.1f}',
                '洗糟前提取系数': '{:.4f}',
                '洗糟水': '{:.1f}',
                '洗糟后提取系数': '{:.4f}',
                '谷物吸收率': '{:.3f}',
                '熬煮蒸发量': '{:.1f}',
                '熬煮后效率': '{:.4f}'
            }, na_rep='---')
            .set_caption("酿造提取系数/效率汇总表（提取系数为小数形式）"))

display(styled_df)
print()

# 统计缺失数据
print(f"\n{'='*100}")
print("📈 数据完整性统计：")
print(f"- 洗糟前提取系数数据: {df['pre_sparge_coefficient'].notna().sum()}/{len(df)} 条")
print(f"- 洗糟后提取系数数据: {df['post_sparge_coefficient'].notna().sum()}/{len(df)} 条")
print(f"- 谷物吸收率数据: {df['grain_absorption_rate'].notna().sum()}/{len(df)} 条")
print(f"- 熬煮后效率数据: {df['post_boil_efficiency'].notna().sum()}/{len(df)} 条")
print(f"{'='*100}\n")


                                           📋 酿造提取系数/效率汇总表                                           



,日期,配方,麦芽重量,糖化水,水料比,PH值,糖化温度,洗糟前提取系数,洗糟水,洗糟后提取系数,谷物吸收率,熬煮蒸发量,熬煮后效率
0,2024-06-29,美式IPA,7.88,24.0,3.23,5.02,68.0,0.7646,7.0,0.7737,0.635,2.0,0.6647
1,2024-08-17,香槟IPA,6.50,20.0,3.08,5.15,63.0,0.5942,10.0,0.6634,0.769,2.0,0.6581
2,2024-09-07,拉格,6.90,22.0,3.19,5.49,65.0,0.7422,10.0,0.9454,0.870,0.5,0.7533
3,2024-10-01,美式IPA,7.05,23.0,3.26,5.51,65.0,0.8431,12.5,0.8651,0.922,2.0,0.6715
4,2024-10-26,混浊IPA,7.65,24.0,3.26,5.57,68.0,0.4729,12.0,---,---,---,---
5,2024-11-16,大麦酒,8.00,24.0,3.00,5.64,65.0,---,6.0,0.7375,0.750,3.0,0.7661
6,2024-12-28,浑浊IPA,7.80,24.0,3.00,5.53,68.0,0.7725,7.0,0.7391,0.897,1.0,0.6542
7,2025-02-03,浑浊IPA,8.00,25.5,3.20,5.32,68.0,0.6042,7.0,0.8281,0.875,1.5,0.6656
8,2025-03-08,浑浊IPA,7.25,23.5,3.20,5.29,67.5,0.5683,7.0,0.7674,0.897,1.0,0.6467
9,2025-04-01,古斯,5.25,18.0,3.40,5.67,65.0,0.8106,9.0,0.9748,0.952,2.0,0.7085




📈 数据完整性统计：
- 洗糟前提取系数数据: 10/11 条
- 洗糟后提取系数数据: 9/11 条
- 谷物吸收率数据: 9/11 条
- 熬煮后效率数据: 10/11 条



In [34]:
# 计算平均值统计
print(f"\n{'='*80}")
print(f"{'📊 提取系数/效率统计分析'.center(80)}")
print(f"{'='*80}\n")

stages = [
    ('洗糟前提取系数', 'pre_sparge_coefficient'),
    ('洗糟后提取系数', 'post_sparge_coefficient'),
    ('谷物吸收率', 'grain_absorption_rate'),
    ('熬煮后效率', 'post_boil_efficiency'),
    ('熬煮蒸发量', 'boil_off')
]

def calculate_trimmed_stats(series):
    """计算去除最大最小2个值后的统计量"""
    valid_data = series.dropna()
    if len(valid_data) <= 4:
        # 如果数据点太少，使用全部数据
        return valid_data.mean() if len(valid_data) > 0 else None, valid_data.std() if len(valid_data) > 0 else None
    else:
        # 去除最大和最小的2个值
        sorted_data = valid_data.sort_values()
        trimmed_data = sorted_data.iloc[2:-2]
        return trimmed_data.mean(), trimmed_data.std()

# 准备统计数据
stats_data = []
for stage_name, col_name in stages:
    valid_data = df[col_name].dropna()
    
    if len(valid_data) > 0:
        # 计算修剪后的平均值和标准差
        trimmed_mean, trimmed_std = calculate_trimmed_stats(df[col_name])
        
        stats_data.append({
            '阶段': stage_name,
            '修剪后平均值': trimmed_mean,
            '修剪后标准差': trimmed_std,
            '原始平均值': valid_data.mean(),
            '最小值': valid_data.min(),
            '最大值': valid_data.max(),
            '样本数': len(valid_data)
        })
    else:
        stats_data.append({
            '阶段': stage_name,
            '修剪后平均值': None,
            '修剪后标准差': None,
            '原始平均值': None,
            '最小值': None,
            '最大值': None,
            '样本数': 0
        })

# 创建统计表格 DataFrame
stats_df = pd.DataFrame(stats_data)
stats_df = stats_df.set_index('阶段')

# 显示美化的统计表格
styled_stats = (stats_df.style.format({
    '修剪后平均值': '{:.4f}',
    '修剪后标准差': '{:.4f}',
    '原始平均值': '{:.4f}',
    '最小值': '{:.4f}',
    '最大值': '{:.4f}',
    '样本数': '{:.0f}'
}, na_rep='---').set_properties(**{'text-align': 'center', 'padding': '10px', 'color': '#495057', 'background-color': 'white'}).set_table_styles([{
    'selector': 'th',
    'props': [('background-color', '#6c757d'),
             ('color', 'white'),
             ('font-weight', 'bold'),
             ('text-align', 'center'),
             ('border', '1px solid #dee2e6'),
             ('padding', '10px')]
}, {
    'selector': 'td',
    'props': [('text-align', 'center'),
             ('border', '1px solid #dee2e6'),
             ('padding', '10px'),
             ('color', '#495057'),
              ('background-color', 'white')]
}, {
    'selector': 'tr:hover',
    'props': [('background-color', '#e9ecef')]
}]).set_caption("各阶段提取系数/效率统计（去除最大最小各2个值）"))

display(styled_stats)
print(f"\n{'='*80}\n")


                                 📊 提取系数/效率统计分析                                  



,修剪后平均值,修剪后标准差,原始平均值,最小值,最大值,样本数
阶段,,,,,,
洗糟前提取系数,0.7077,0.0848,0.6941,0.4729,0.8431,10
洗糟后提取系数,0.7947,0.0509,0.8105,0.6634,0.9748,9
谷物吸收率,0.8616,0.0531,0.8407,0.6345,0.9524,9
熬煮后效率,0.6869,0.0371,0.6988,0.6467,0.7993,10
熬煮蒸发量,1.7000,0.4472,1.6667,0.5000,3.0000,9


In [35]:
# 计算修剪后的平均值并输出到CSV

print("\n" + "="*80)
print("✅ 酿造提取系数/效率分析完成")
print("="*80)

# 计算五个关键指标的修剪后平均值
metrics = [
    ('pre_sparge_coefficient', 'pre_sparge_coefficient', '洗糟前提取系数'),
    ('post_sparge_coefficient', 'post_sparge_coefficient', '洗糟后提取系数'),
    ('grain_absorption_rate', 'grain_absorption_rate', '谷物吸收率'),
    ('post_boil_efficiency', 'post_boil_efficiency', '熬煮后效率'),
    ('boil_off_volume', 'boil_off', '熬煮蒸发量')
]

# 创建结果字典
results = {}
metric_names_cn = []

print("\n📊 修剪后平均值（去除最大最小各2个值）：")
print("-"*80)

for eng_key, col_name, cn_name in metrics:
    trimmed_mean, _ = calculate_trimmed_stats(df[col_name])
    if trimmed_mean is not None:
        results[eng_key] = trimmed_mean
        metric_names_cn.append(cn_name)
        print(f"{cn_name:>12}: {trimmed_mean:>8.4f}")

# 导出到CSV
output_file = 'brewing_metrics_summary.csv'

# 创建DataFrame并保存
result_df = pd.DataFrame([results])
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n💾 数据已导出到 {output_file}")
print(f"   包含字段: {', '.join(results.keys())}")
print("="*80)


✅ 酿造提取系数/效率分析完成

📊 修剪后平均值（去除最大最小各2个值）：
--------------------------------------------------------------------------------
     洗糟前提取系数:   0.7077
     洗糟后提取系数:   0.7947
       谷物吸收率:   0.8616
       熬煮后效率:   0.6869
       熬煮蒸发量:   1.7000

💾 数据已导出到 brewing_metrics_summary.csv
   包含字段: pre_sparge_coefficient, post_sparge_coefficient, grain_absorption_rate, post_boil_efficiency, boil_off_volume
